# HW1 STARTER NOTEBOOK

## PART 1: UNSUPERVISED LEARNING

In [1]:
import pandas as pd

### LOAD DATA

In [2]:
df = pd.read_csv("data/data_reduced_2025_flavors_of_cacao.csv", index_col=0)

ignore_cols = [0,1,2,3,4,5,6,7,15]
df.drop(df.columns[ignore_cols], axis=1).columns

Index(['B', 'C', 'L', 'S', 'S*', 'Sa', 'V', 'nuts', 'tangy', 'bland',
       'chemical', 'orange', 'green', 'rubber', 'marshmallow', 'basic cocoa',
       'burnt', 'muted', 'citrus', 'coarse', 'red berry', 'off notes',
       'black pepper', 'bold', 'mild bitter', 'hammy', 'strawberry', 'smooth',
       'dirty', 'rubbery', 'mild fruit', 'melon', 'metallic', 'nut', 'honey',
       'tobacco', 'smoke', 'vegetal', 'pungent', 'off', 'cherry', 'smokey',
       'acidic', 'astringent', 'banana', 'brownie', 'oily', 'dairy', 'complex',
       'rich cocoa', 'fruity', 'dry', 'gritty', 'spice', 'grassy', 'tart',
       'bitter', 'caramel', 'rich', 'coffee', 'dried fruit', 'sticky',
       'vanilla', 'woody', 'molasses', 'fruit', 'intense', 'spicy', 'sour',
       'floral', 'sandy', 'fatty', 'creamy', 'earthy', 'roasty', 'nutty',
       'cocoa', 'sweet'],
      dtype='object')

### PREPARE DATAFRAME

In [3]:
df_cluster = df.drop(df.columns[ignore_cols], axis=1)

In [4]:
df_cluster

,B,C,L,S,S*,Sa,V,nuts,tangy,bland,...,sour,floral,sandy,fatty,creamy,earthy,roasty,nutty,cocoa,sweet
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2785,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2786,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2787,1,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### PERFORM CLUSTERING

In [5]:
from sklearn.cluster import KMeans

# PERFORM KMEANS K=2
kmeans        = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(df_cluster)

# ASSIGN CLUSTER NUMBER IN DF
df['cluster'] = kmeans.predict(df_cluster)

In [6]:
def generate_cluster_report(k, d):
    print(f"CLUSTER {k} SUMMARY")

    print(f"Mean Rating: {d['Rating'].mean():.3f}")

    print(f"Mean Cocoa %: {d['Cocoa Percent'].mean()*100:.2f}%")
    
    print("Median review date:", int(d['Review Date'].median()) )
    
    print("Top 5 Bean Origins")
    for c, v in d['Country of Bean Origin'].value_counts(ascending=False).nlargest(5).items():
        print("    ", c, "...", v)

    print("Top 5 Company Locations")
    for c, v in d['Company Location'].value_counts(ascending=False).nlargest(5).items():
        print("    ", c, "...", v)

### REPORT ON CLUSTERS

In [7]:
# generate report for data assigned cluster 0
generate_cluster_report( 0, df.iloc[:,:8].loc[df.cluster==0] )
print("\n====\n")

# generate the report for data assigned cluster 1
generate_cluster_report( 1, df.iloc[:,:8].loc[df.cluster==1] )

CLUSTER 0 SUMMARY
Mean Rating: 3.213
Mean Cocoa %: 71.13%
Median review date: 2015
Top 5 Bean Origins
     Venezuela ... 192
     Peru ... 181
     Ecuador ... 152
     Dominican Republic ... 138
     Madagascar ... 122
Top 5 Company Locations
     U.S.A. ... 723
     France ... 167
     Canada ... 138
     U.K. ... 108
     Belgium ... 71

====

CLUSTER 1 SUMMARY
Mean Rating: 3.166
Mean Cocoa %: 72.46%
Median review date: 2015
Top 5 Bean Origins
     Dominican Republic ... 107
     Peru ... 95
     Ecuador ... 78
     Venezuela ... 75
     Madagascar ... 72
Top 5 Company Locations
     U.S.A. ... 537
     Canada ... 50
     Italy ... 40
     U.K. ... 27
     France ... 20


In [8]:
'''
    Return the domininant characteristics of the cluster.

        thresh : the number of characteristics to limit to
'''
def get_cluster_characteristics(centers, columns, thresh=10):
    return \
        pd.DataFrame(centers, index=columns).nlargest(thresh, columns=0).index.to_list()

In [9]:
# top 10 dominant characteristics for cluster 0
get_cluster_characteristics(kmeans.cluster_centers_[0], df_cluster.columns)

['C', 'B', 'S', 'L', 'V', 'sweet', 'nutty', 'cocoa', 'fatty', 'creamy']

In [10]:
# top 10 dominant characteristics for cluster 1
get_cluster_characteristics(kmeans.cluster_centers_[1], df_cluster.columns)

['B',
 'S',
 'cocoa',
 'roasty',
 'earthy',
 'nutty',
 'intense',
 'sour',
 'sweet',
 'spicy']

### SUMMARIZE CLUSTER CHARACTERISTICS BELOW

## PART 2: UNSUPERVISED LEARNING

### LOAD TRAINING DATA

In [11]:
df_train = pd.read_csv("data/train.csv", index_col=0)

### SHOW DATA

In [12]:
df_train

,B,C,L,S,S*,Sa,V,nuts,tangy,bland,...,floral,sandy,fatty,creamy,earthy,roasty,nutty,cocoa,sweet,cluster
615,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1807,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
125,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1576,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
231,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2629,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1959,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2272,1,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
737,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2482,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0


### APPLY BERNOULLI NAIVE BAYES

In [13]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()

X = df_train.iloc[:,:-1]
Y = df_train.iloc[:,-1:]['cluster'].values

clf.fit(X, Y)

BernoulliNB()

### LOAD AND ASSIGN TEST DATA

In [14]:
df_test = pd.read_csv("data/test.csv", index_col=0)
df_test['cluster_predict'] = clf.predict(df_test.iloc[:,:-1])

### GET MODEL ACCURACY 

In [15]:
df_test.iloc[:,-2:].query('cluster != cluster_predict').shape[0] / df_test.shape[0]

0.4906407487401008